In [1]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from scipy.spatial.distance import pdist, squareform

In [2]:
#presumed_residence = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/presumed_res_and_call_loc_without_repetition.csv')
#presumed_residence.sample(15)
df_user_with_residence = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/df_main.csv')
df_user_with_residence = df_user_with_residence.drop('Unnamed: 0', 1)
df_user_with_residence

,USER,ANTENNA_ID,PRESUMED_RESIDENCE
0,00003F26576D1122848B6F3CEEC250E5,SJDR5,SJDR5
1,00003F26576D1122848B6F3CEEC250E5,SJDR5,SJDR5
2,00003F26576D1122848B6F3CEEC250E5,SJDR5,SJDR5
3,00003F26576D1122848B6F3CEEC250E5,SJDR5,SJDR5
4,00003F26576D1122848B6F3CEEC250E5,SJDR5,SJDR5
...,...,...,...
543587,FFFF6761884073022FB5DFE1B6F54DE3,RT,RT
543588,FFFF6761884073022FB5DFE1B6F54DE3,RT,RT
543589,FFFF6761884073022FB5DFE1B6F54DE3,RT,RT
543590,FFFF6761884073022FB5DFE1B6F54DE3,RT,RT


In [ ]:
antennas_with_id = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/antenas_with_id.csv', index_col=0)
antennas_with_id.sample(15)

In [ ]:
antennas_with_id.groupby(['ANTENNA_ID']).min().reset_index()

In [ ]:
antennas_and_lat_long = antennas_with_id.groupby('ANTENNA_ID').agg({'LAT': min, 'LONG':min}).reset_index()
antennas_and_lat_long

# Calculando a distância entre as antenas

In [ ]:
def dist(x, y):
    
    lat1 = radians(x[0])
    lon1 = radians(x[1])
    lat2 = radians(y[0])
    lon2 = radians(y[1])
    
    R = 6373.0
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat/2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon/2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    
    return round(distance, 4)

In [ ]:
distances = pdist(antennas_and_lat_long[['LAT', 'LONG']].values, metric=dist)

points = antennas_and_lat_long['ANTENNA_ID'].values

result = pd.DataFrame(squareform(distances), columns=points, index=points)

result

In [ ]:
melt_df = pd.melt(result.reset_index(), id_vars='index')
melt_df.rename(columns={'index': 'ANTENNA_ID', 'variable': 'PRESUMED_RESIDENCE', 'value': 'DISTANCE'}, inplace=True)

melt_df

In [ ]:
df_main = pd.merge(df_user_with_residence, melt_df, left_on=['ANTENNA_ID', 'PRESUMED_RESIDENCE'], right_on=['ANTENNA_ID', 'PRESUMED_RESIDENCE'])
df_main.sample(10)

In [ ]:
df_final = df_main.groupby(['USER', 'PRESUMED_RESIDENCE']).agg({'ANTENNA_ID': list, 'DISTANCE': sum}).reset_index()
df_final.rename(columns={'ANTENNA_ID':'CALL_LOC', 'DISTANCE': 'TRAVELLED_DISTANCE'}, inplace=True)



In [ ]:
df_final.sample(10)